In [1]:
import pandas as pd
import numpy as np
import config
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
specialite=10
year=21
method="PS"
dept=6

sql = f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id 
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join iris.iris i on i.code=an.iris
where sp.specialite_id=10
and tds.date_source_id >= 2100 and  tds.date_source_id < 2200
and ar.dept_id=6
group by ps.id, c.id, an.id, i.id
"""

if method=="PA":
    sql = f"""
    select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from personne_activite pa
    join personne_activite_diplome pad on pad.personne_activite_id=pa.id
    join diplome d on pad.diplome_id=d.id
    join profession_diplome pd on pd.diplome_id=d.id
    join profession p on pd.profession_id=p.id
    join specialite_profession sp on sp.profession_id=p.id
    join pa_adresse_norm_date_source pands ON pands.personne_activite_id = pa.id
    join adresse_norm an on pands.adresse_norm_id=an.id
    join iris.iris i on i.code=an.iris
    where sp.specialite_id=10
    and pands.date_source_id/100=21
    and an.dept_id=6
    group by pa.id, an.id, i.id
    """
elif method=="RPPS":
    sql = f"""
    select personne.id, personne.nom, personne.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from personne
    join diplome_obtenu dio on dio.personne_id = personne.id
    join diplome d on d.id=dio.diplome_id
    join profession_diplome pd on pd.diplome_id=d.id
    join profession p on pd.profession_id=p.id
    join specialite_profession sp on sp.profession_id=p.id
    join coord c on c.personne_id=personne.id
    join adresse_norm an on c.adresse_norm_id=an.id
    join iris.iris i on i.code=an.iris
    where sp.specialite_id=10
    and personne.date_maj > '2022-01-01' and date_effet < '2022-01-01'
    and an.dept_id=6
    group by personne.id, an.id, i.id
    """

print(sql)
gene = pd.read_sql(sql, config.connection_string)
# 1286
# 50488 gene en 2023
# 47058 en 2024
gene.style.set_table_attributes('style="font-size: 10px;height:10px;"')
# gene = pd.read_stata("Etude 06/Gene06.dta")
gene #1289


select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id 
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join iris.iris i on i.code=an.iris
where sp.specialite_id=10
and tds.date_source_id >= 2100 and  tds.date_source_id < 2200
and ar.dept_id=6
group by ps.id, c.id, an.id, i.id



,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat
0,16126,LEONARDI,DOMINIQUE,6,1415,60881903,7.255881,43.709653
1,16130,BENSUSSAN,JEAN,6,92990,60882801,7.227639,43.676026
2,16131,LARA,JEANNINE,6,92963,60880202,7.282865,43.695573
3,16132,GUEGAN,JEAN CLAUDE,6,1425,60270102,7.145627,43.657352
4,16148,CROQUET,PATRICE,6,1433,60690109,6.922279,43.651604
...,...,...,...,...,...,...,...,...
1280,155689,SAROUX,DELPHINE,6,2305,60882401,7.258125,43.695605
1281,155690,PERADOTTO,CYRIL,6,2866,60290121,6.988390,43.550361
1282,155692,DONADIO,MANON,6,77494,61490102,7.313438,43.743446
1283,155827,BOUCHET BADEL,MARIE,6,2328,60690102,6.914032,43.646318


In [3]:
gene["id"].nunique() # 1189 sql 1203

1203

In [4]:
gene["b"] = gene["id"].astype(str) + "_" + gene["lat"].astype(str) + "_" + gene["lon"].astype(str)
gene["b"].nunique() #1289 #sql 1274

1273

In [5]:
gene = gene.sort_values(by='id')
gene["un"] = gene.groupby("id")["b"].transform("nunique")
gene["un"].value_counts(normalize=True) # 85 12 1

un
1    0.893385
2    0.094163
3    0.009339
4    0.003113
Name: proportion, dtype: float64

In [6]:
gene['weight'] = 1 / gene['un']
gene.head(10)

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,b,un,weight
0,16126,LEONARDI,DOMINIQUE,6,1415,60881903,7.255881,43.709653,16126_43.709653_7.255881,1,1.0
1,16130,BENSUSSAN,JEAN,6,92990,60882801,7.227639,43.676026,16130_43.676026_7.227639,1,1.0
2,16131,LARA,JEANNINE,6,92963,60880202,7.282865,43.695573,16131_43.695573_7.282865,1,1.0
3,16132,GUEGAN,JEAN CLAUDE,6,1425,60270102,7.145627,43.657352,16132_43.657352_7.145627,1,1.0
4,16148,CROQUET,PATRICE,6,1433,60690109,6.922279,43.651604,16148_43.651604_6.922279,1,1.0
5,16151,DESGATS,MARC,6,1436,60300107,7.005844,43.579045,16151_43.579045_7.005844,1,1.0
6,16153,DUBOIS,MICHEL,6,1438,61360000,7.449651,43.877046,16153_43.877046_7.449651,1,1.0
7,16159,GIDEL,PIERRE JEAN,6,1466,60882601,7.242220,43.689322,16159_43.689322_7.24222,1,1.0
8,16160,MASCHINO,JEAN MARC,6,1467,60290102,7.037066,43.546353,16160_43.546353_7.037066,1,1.0
9,16161,SARDOU,YVES,6,1468,60830205,7.492397,43.791639,16161_43.791639_7.492397,1,1.0


In [7]:
gene = gene.sort_values(by='iris')
gene["NB"] = gene.groupby("iris")["weight"].transform("sum")
gene
gene[gene["iris"]==61290000]

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,b,un,weight,NB
53,16295,PARODI CARIZZONI,CATHERINE,6,264607,61290000,7.103691,44.083070,16295_44.08307_7.103691,1,1.0,2.5
1084,132354,BACCA,JEAN PIERRE,6,88507,61290000,7.104934,44.083809,132354_44.0838086_7.1049345,2,0.5,2.5
180,16554,AUGIER,BERNARD,6,82001,61290000,7.106376,44.081912,16554_44.081912_7.106376,2,0.5,2.5
179,16554,AUGIER,BERNARD,6,270682,61290000,7.106376,44.081912,16554_44.081912_7.106376,2,0.5,2.5


In [8]:
gene = gene.drop_duplicates(subset=['iris', 'NB'])
# gene.to_csv("nbgpiris2022.csv", index=False)
gene

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,b,un,weight,NB
123,16425,DAEMERS,YVES,6,92989,60040102,7.072160,43.596934,16425_43.596934_7.07216,1,1.0,2.000000
904,113924,DIAMANT,GUY,6,77467,60040103,7.137517,43.567521,113924_43.567521_7.137517,1,1.0,1.000000
109,16404,MANDRILE,DIDIER,6,1651,60040104,7.092388,43.604533,16404_43.604533_7.092388,1,1.0,1.000000
31,16222,NICASTRO,SERGE,6,92975,60040105,7.117538,43.588105,16222_43.588105_7.117538,1,1.0,2.000000
1179,144024,FRICKEY,SARAH,6,2734,60040106,7.112369,43.606074,144024_43.606074_7.112369,1,1.0,8.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
607,17388,ROUSSEAU,FRANCOISE,6,2635,61610103,7.123625,43.658047,17388_43.658047_7.123625,1,1.0,1.000000
1018,125002,CHEMOUNI,KARINE,6,84186,61610104,7.142204,43.642154,125002_43.642154_7.142204,1,1.0,3.000000
984,121356,DETANNE,SOLINE,6,271634,61610105,7.141885,43.640635,121356_43.640635_7.141885,2,0.5,5.000000
301,16814,DI VINCENZO,DOMINIQUE,6,273951,61620000,7.612673,44.063665,16814_44.063665_7.612673,2,0.5,0.500000


In [9]:
min=30
dept=6
sql = f"""
(select iris.id "IRIS1", iris.id "IRIS2", 0 "KM", 0 "TIME" from iris.iris
where id / 10000000 = {dept}

union

select iris_id_from as "IRIS1", iris_id_to "IRIS2", route_km "KM", route_min "TIME" from iris.iris_matrix
where (iris_id_from/10000000={dept} or iris_id_to/10000000={dept})
and route_min <= {min}) order by "IRIS1", "IRIS2"
"""

# od06sql = pd.read_sql(sql, config.connection_string)
# od06sql.to_csv("od_06_sql.csv")
# od = od06sql
# od06sql

# od = pd.read_stata("Etude 06/ODFinale.dta")
# od = od[od["TIME"] <= 30]
# od #80712
# od

od = pd.read_csv("od_06_sql.csv")
od.style.set_table_attributes('style="font-size: 10px;height:10px;"')
od


,IRIS1,IRIS2,KM,TIME
0,40080000,60530000,17,22
1,40080000,60990000,21,24
2,40390000,61540000,24,30
3,40420000,60530000,11,19
4,40420000,60710000,22,30
...,...,...,...,...
117764,831380000,60850103,34,29
117765,831380000,60950101,18,19
117766,831380000,60950102,19,22
117767,831380000,60950103,18,20


In [10]:
pd.options.mode.copy_on_write = True
od["iris"] = od["IRIS2"].astype("int64")
od["iris"].nunique() # 500

566

In [11]:
from sqlalchemy import text
yy=20 if year==20 else 21
sql = f"""
select pi.iris, c.code "COM", i.type "TYP_IRIS", pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4559, pi.pop6074, pi.pop75p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '{dept:02}%'
and year={yy}
"""
print(sql)
pop_iris = pd.read_sql(text(sql), config.connection_string)
# pop_iris = pd.read_stata("Etude 06/popIRIS.dta")
# /!\ Il y a 1 trou dans les pop à refaire
pop_iris


select pi.iris, c.code "COM", i.type "TYP_IRIS", pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4559, pi.pop6074, pi.pop75p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '06%'
and year=21



,iris,COM,TYP_IRIS,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4559,pop6074,pop75p
0,060120105,06012,A,68.356436,55.185368,179.895476,266.832339,226.432893,571.248108,798.278095,470.022115,264.724335
1,061230108,06123,A,88.787020,75.776145,135.840215,156.862889,129.702563,508.721610,405.595984,474.739115,360.374803
2,060500000,06050,C,14.188627,14.119226,40.517897,47.603916,17.493201,71.924387,136.081254,95.820751,43.938756
3,061530000,06153,C,18.020482,23.026074,44.052137,67.079572,41.031287,120.144366,192.236933,209.260823,74.092761
4,060720000,06072,C,0.000000,6.761905,4.829932,4.829932,3.863946,21.251701,36.707483,33.809524,25.115646
...,...,...,...,...,...,...,...,...,...,...,...,...
495,060881606,06088,A,56.982735,81.433187,104.491854,148.417994,461.133207,399.230815,415.984718,505.992806,327.462243
496,060881701,06088,A,24.463401,43.859704,85.003020,126.388729,282.367384,250.163180,346.730601,253.225570,185.286878
497,060881804,06088,A,85.043040,83.274932,124.543590,225.112876,294.722303,673.422185,532.846040,373.835644,306.477529
498,060881805,06088,A,65.721430,54.233925,76.225674,153.780470,257.227060,430.652161,379.444333,357.451476,287.465848


In [12]:
pop_iris["iris"] = pd.to_numeric(pop_iris["iris"], errors="coerce")
od = od.merge(pop_iris, on="iris", how="left", suffixes=('', ''))
od

,IRIS1,IRIS2,KM,TIME,iris,COM,TYP_IRIS,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4559,pop6074,pop75p
0,40080000,60530000,17,22,60530000,06053,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794
1,40080000,60990000,21,24,60990000,06099,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,364.380158,359.428455,210.546497
2,40390000,61540000,24,30,61540000,06154,C,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,97.576377,115.832831,64.096011
3,40420000,60530000,11,19,60530000,06053,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794
4,40420000,60710000,22,30,60710000,06071,C,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,117.816891,152.888385,89.978360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,60850103,06085,A,57.428416,72.207652,135.459522,194.409719,141.793750,318.526897,569.348679,564.134832,403.377763
117765,831380000,60950101,18,19,60950101,06095,A,77.910796,87.987431,210.729551,255.894450,155.595176,478.030563,686.597900,648.683485,415.708340
117766,831380000,60950102,19,22,60950102,06095,A,67.975915,77.960010,155.903079,226.090400,145.568264,401.767992,533.594615,490.010580,296.409159
117767,831380000,60950103,18,20,60950103,06095,A,46.976690,68.925850,125.728002,224.736092,103.186116,259.487677,570.055119,581.257106,330.447800


In [13]:
od["WexpGP"] = np.exp(-0.12 * od["TIME"])
# od.to_csv("popiris2022.csv", index=False)
od.head(10)



,IRIS1,IRIS2,KM,TIME,iris,COM,TYP_IRIS,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4559,pop6074,pop75p,WexpGP
0,40080000,60530000,17,22,60530000,06053,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.071361
1,40080000,60990000,21,24,60990000,06099,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,364.380158,359.428455,210.546497,0.056135
2,40390000,61540000,24,30,61540000,06154,C,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,97.576377,115.832831,64.096011,0.027324
3,40420000,60530000,11,19,60530000,06053,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.102284
4,40420000,60710000,22,30,60710000,06071,C,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,117.816891,152.888385,89.978360,0.027324
5,40420000,60990000,17,24,60990000,06099,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,364.380158,359.428455,210.546497,0.056135
6,40420000,61240000,15,30,61240000,06124,C,2.557377,0.852459,0.000000,0.000000,0.852459,6.819672,7.672131,17.901639,14.491803,0.027324
7,40430000,60020000,15,20,60020000,06002,C,0.944444,0.000000,0.944444,0.000000,4.722222,2.833333,15.111111,15.111111,9.444444,0.090718
8,40430000,60240000,8,11,60240000,06024,C,4.380952,5.476190,4.380952,16.428571,3.285714,19.714286,31.761905,48.190476,38.333333,0.267135
9,40430000,60450000,18,23,60450000,06045,C,2.710965,0.903655,0.903655,1.807310,4.518275,11.840499,14.458479,23.495029,15.362134,0.063292


In [14]:
# od = od.sort_values(by='IRIS2')
# P19_POP0002 P19_POP0305 P19_POP0610 P19_POP1117 P19_POP1824 P19_POP2539 P19_POP4559 P19_POP6074 P19_POP75P
pop_cols = ["pop0002", "pop0305", "pop0610", "pop1117", "pop1824", "pop2539", "pop4559", "pop6074", "pop75p"]
weights = [0.759201627]*4 + [0.784999993, 0.915, 1.05, 1.4, 1.3] # cf doc 070-analyse... p8
for col in pop_cols:
    od[col] = pd.to_numeric(od[col], errors="coerce")
od["POPGP"] = sum(w * od[c] for w, c in zip(weights, pop_cols))
# od.to_csv("popiris2022.csv", index=False)
od.head(10)

,IRIS1,IRIS2,KM,TIME,iris,COM,TYP_IRIS,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4559,pop6074,pop75p,WexpGP,POPGP
0,40080000,60530000,17,22,60530000,06053,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.071361,154.065603
1,40080000,60990000,21,24,60990000,06099,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,364.380158,359.428455,210.546497,0.056135,1795.953379
2,40390000,61540000,24,30,61540000,06154,C,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,97.576377,115.832831,64.096011,0.027324,500.849234
3,40420000,60530000,11,19,60530000,06053,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.102284,154.065603
4,40420000,60710000,22,30,60710000,06071,C,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,117.816891,152.888385,89.978360,0.027324,614.865324
5,40420000,60990000,17,24,60990000,06099,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,364.380158,359.428455,210.546497,0.056135,1795.953379
6,40420000,61240000,15,30,61240000,06124,C,2.557377,0.852459,0.000000,0.000000,0.852459,6.819672,7.672131,17.901639,14.491803,0.027324,61.455310
7,40430000,60020000,15,20,60020000,06002,C,0.944444,0.000000,0.944444,0.000000,4.722222,2.833333,15.111111,15.111111,9.444444,0.090718,57.033492
8,40430000,60240000,8,11,60240000,06024,C,4.380952,5.476190,4.380952,16.428571,3.285714,19.714286,31.761905,48.190476,38.333333,0.267135,194.550040
9,40430000,60450000,18,23,60450000,06045,C,2.710965,0.903655,0.903655,1.807310,4.518275,11.840499,14.458479,23.495029,15.362134,0.063292,87.228514


In [15]:
# KM iris COM TYP_IRIS MODIF_IRIS LAB_IRIS P19_POP P19_POP0002 P19_POP0305 P19_POP0610 P19_POP1117 P19_POP1824 P19_POP2539 P19_POP4054 P19_POP5564 P19_POP6579 P19_POP80P P19_POP0014 P19_POP1529 P19_POP3044 P19_POP4559 P19_POP6074 P19_POP75P P19_POP0019 P19_POP2064 P19_POP65P P19_POPH P19_H0014 P19_H1529 P19_H3044 P19_H4559 P19_H6074 P19_H75P P19_H0019 P19_H2064 P19_H65P P19_POPF P19_F0014 P19_F1529 P19_F3044 P19_F4559 P19_F6074 P19_F75P P19_F0019 P19_F2064 P19_F65P _merge WexpGP POPGP
od = od.drop(["KM","iris","COM"], axis=1)
for col in od.columns:
    if col.startswith("p20"):
        od = od.drop(col, axis=1)
# od2 = od.drop(["WexpGP","POPGP", "TYP_IRIS"], axis=1)
# # od2.to_csv("ODgeneralistes06.csv", index=False)
# od2
od

,IRIS1,IRIS2,TIME,TYP_IRIS,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4559,pop6074,pop75p,WexpGP,POPGP
0,40080000,60530000,22,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.071361,154.065603
1,40080000,60990000,24,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,364.380158,359.428455,210.546497,0.056135,1795.953379
2,40390000,61540000,30,C,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,97.576377,115.832831,64.096011,0.027324,500.849234
3,40420000,60530000,19,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.102284,154.065603
4,40420000,60710000,30,C,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,117.816891,152.888385,89.978360,0.027324,614.865324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,29,A,57.428416,72.207652,135.459522,194.409719,141.793750,318.526897,569.348679,564.134832,403.377763,0.030807,2663.613353
117765,831380000,60950101,19,A,77.910796,87.987431,210.729551,255.894450,155.595176,478.030563,686.597900,648.683485,415.708340,0.102284,3209.257599
117766,831380000,60950102,22,A,67.975915,77.960010,155.903079,226.090400,145.568264,401.767992,533.594615,490.010580,296.409159,0.071361,2514.314725
117767,831380000,60950103,20,A,46.976690,68.925850,125.728002,224.736092,103.186116,259.487677,570.055119,581.257106,330.447800,0.090718,2514.398596


In [16]:
od["iris"] = od["IRIS1"]
od = od.sort_values(by='IRIS1')
nbgp = gene
od = od.merge(nbgp, on="iris", how="left", suffixes=('', ''))
od["NB"].unique()
od

,IRIS1,IRIS2,TIME,TYP_IRIS,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4559,pop6074,pop75p,WexpGP,POPGP,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,b,un,weight,NB
0,40080000,60530000,22,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.071361,154.065603,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40080000,60990000,24,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,364.380158,359.428455,210.546497,0.056135,1795.953379,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40390000,61540000,30,C,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,97.576377,115.832831,64.096011,0.027324,500.849234,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40420000,60530000,19,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.102284,154.065603,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40420000,60710000,30,C,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,117.816891,152.888385,89.978360,0.027324,614.865324,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60300110,29,A,76.253614,76.029070,77.647792,221.672941,196.790876,313.104831,582.212824,408.666186,372.889243,0.030807,2452.041947,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117765,831380000,60300112,30,A,75.056425,83.641050,134.623635,145.485073,163.300967,506.912628,508.707539,410.711732,319.372839,0.027324,2449.482713,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117766,831380000,60300113,29,A,74.155648,110.295080,103.985776,182.164445,187.950284,561.740578,535.862118,513.437498,367.032350,0.030807,2777.424384,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117767,831380000,60690101,27,A,119.403068,152.696795,255.661696,349.884305,253.288995,715.102826,689.512312,545.716376,233.272236,0.039164,3310.705874,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
od["NB"] = od["NB"].fillna(0)
od = od.sort_values(by=['IRIS1', "IRIS2"])
# od.to_csv("IRIS2022.csv", index=False)
od

,IRIS1,IRIS2,TIME,TYP_IRIS,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4559,pop6074,pop75p,WexpGP,POPGP,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,b,un,weight,NB
0,40080000,60530000,22,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.071361,154.065603,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,40080000,60990000,24,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,364.380158,359.428455,210.546497,0.056135,1795.953379,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,40390000,61540000,30,C,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,97.576377,115.832831,64.096011,0.027324,500.849234,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,40420000,60530000,19,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.102284,154.065603,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,40420000,60710000,30,C,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,117.816891,152.888385,89.978360,0.027324,614.865324,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117737,831380000,60850103,29,A,57.428416,72.207652,135.459522,194.409719,141.793750,318.526897,569.348679,564.134832,403.377763,0.030807,2663.613353,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
117738,831380000,60950101,19,A,77.910796,87.987431,210.729551,255.894450,155.595176,478.030563,686.597900,648.683485,415.708340,0.102284,3209.257599,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
117739,831380000,60950102,22,A,67.975915,77.960010,155.903079,226.090400,145.568264,401.767992,533.594615,490.010580,296.409159,0.071361,2514.314725,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
117744,831380000,60950103,20,A,46.976690,68.925850,125.728002,224.736092,103.186116,259.487677,570.055119,581.257106,330.447800,0.090718,2514.398596,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [18]:
od = od.sort_values(by='IRIS1')
od["wpop"] = od["WexpGP"] * od["POPGP"]
od["swpop"] = od.groupby("IRIS1")["wpop"].transform("sum")
od["R"] = od["NB"] / (od["swpop"] / 100000)
od.head(10)

,IRIS1,IRIS2,TIME,TYP_IRIS,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4559,pop6074,pop75p,WexpGP,POPGP,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,b,un,weight,NB,wpop,swpop,R
0,40080000,60530000,22,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.071361,154.065603,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.994317,111.809734,0.0
1,40080000,60990000,24,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,364.380158,359.428455,210.546497,0.056135,1795.953379,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,100.815417,111.809734,0.0
2,40390000,61540000,30,C,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,97.576377,115.832831,64.096011,0.027324,500.849234,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.685065,13.685065,0.0
3,40420000,60530000,19,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.102284,154.065603,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.758478,135.053492,0.0
4,40420000,60710000,30,C,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,117.816891,152.888385,89.978360,0.027324,614.865324,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,16.800409,135.053492,0.0
5,40420000,60990000,24,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,364.380158,359.428455,210.546497,0.056135,1795.953379,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,100.815417,135.053492,0.0
6,40420000,61240000,30,C,2.557377,0.852459,0.000000,0.000000,0.852459,6.819672,7.672131,17.901639,14.491803,0.027324,61.455310,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.679188,135.053492,0.0
7,40430000,61310000,28,C,0.000000,1.000000,2.000000,1.000000,2.000000,5.000000,13.000000,11.000000,11.000000,0.034735,52.531806,40430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.824706,152.476357,0.0
8,40430000,61160000,26,C,5.679112,2.754914,5.542487,12.252333,5.779466,18.246118,56.507006,61.225129,34.376207,0.044157,230.881667,40430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.195081,152.476357,0.0
9,40430000,60870000,22,C,0.000000,0.000000,0.975610,0.975610,3.902439,2.926829,9.756098,10.731707,8.780488,0.071361,43.905759,40430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.133171,152.476357,0.0


In [19]:
rgp = od[od["IRIS1"] == od["IRIS2"]]
rgp = rgp[["IRIS1", "TYP_IRIS", "POPGP", "NB", "R"]].copy()
rgp.head(10)

,IRIS1,TYP_IRIS,POPGP,NB,R
81,60010000,C,92.810629,0.000000,0.000000
92,60020000,C,57.033492,0.000000,0.000000
110,60030000,C,622.897211,0.000000,0.000000
359,60040101,C,1644.085142,0.000000,0.000000
713,60040102,A,2409.644871,2.000000,1.435413
1017,60040103,A,864.660366,1.000000,1.271532
1356,60040104,A,2172.216717,1.000000,0.611100
1700,60040105,A,3992.283201,2.000000,1.474023
2044,60040106,A,2719.305357,8.666667,6.604219
2387,60040107,A,3038.758340,4.000000,2.863398


In [20]:
rgp["IRIS2"] = rgp["IRIS1"]
# rgp.to_csv("RGP2022.csv", index=False)
od = od.merge(rgp, on="IRIS2", suffixes=("", "_dest"))
od.head(10)

,IRIS1,IRIS2,TIME,TYP_IRIS,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4559,pop6074,pop75p,WexpGP,POPGP,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,b,un,weight,NB,wpop,swpop,R,IRIS1_dest,TYP_IRIS_dest,POPGP_dest,NB_dest,R_dest
0,40080000,60530000,22,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.071361,154.065603,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.994317,111.809734,0.000000,60530000,C,154.065603,0.0,0.0
1,40420000,60530000,19,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.102284,154.065603,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.758478,135.053492,0.000000,60530000,C,154.065603,0.0,0.0
2,40760000,60530000,19,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.102284,154.065603,40760000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.758478,721.818155,0.000000,60530000,C,154.065603,0.0,0.0
3,40900000,60530000,28,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.034735,154.065603,40900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5.351509,54.423640,0.000000,60530000,C,154.065603,0.0,0.0
4,41740000,60530000,20,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.090718,154.065603,41740000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.976516,142.138029,0.000000,60530000,C,154.065603,0.0,0.0
5,42020000,60530000,9,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.339596,154.065603,42020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,52.319989,319.041362,0.000000,60530000,C,154.065603,0.0,0.0
6,60530000,60530000,0,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,1.000000,154.065603,60530000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,154.065603,366.342257,0.000000,60530000,C,154.065603,0.0,0.0
7,60710000,60530000,18,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.115325,154.065603,60710000,16333.0,BOFFY,FRANCOIS,6.0,1937.0,6.852913,44.091130,16333_44.09113_6.852913,1.0,1.0,1.0,17.767634,1107.943375,90.257320,60530000,C,154.065603,0.0,0.0
8,60940000,60530000,25,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.049787,154.065603,60940000,91295.0,GIRAUD,ROLAND,6.0,63080.0,6.932793,44.094193,91295_44.094193_6.932793,1.0,1.0,1.0,7.670475,1326.103152,75.408915,60530000,C,154.065603,0.0,0.0
9,60990000,60530000,26,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.044157,154.065603,60990000,102698.0,DIETZ,JEAN PAUL,6.0,98683.0,6.898646,43.953725,102698_43.953725_6.898646,1.0,1.0,1.0,6.803101,2139.465670,46.740642,60530000,C,154.065603,0.0,0.0


In [21]:
od = od.sort_values(by=['IRIS1', "IRIS2"]).copy()
od["Ap"] = od["WexpGP"] * od["R_dest"]
od["APL"] = od.groupby("IRIS1")["Ap"].transform("sum")
od

,IRIS1,IRIS2,TIME,TYP_IRIS,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4559,pop6074,pop75p,WexpGP,POPGP,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,b,un,weight,NB,wpop,swpop,R,IRIS1_dest,TYP_IRIS_dest,POPGP_dest,NB_dest,R_dest,Ap,APL
0,40080000,60530000,22,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.071361,154.065603,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.994317,111.809734,0.0,60530000,C,154.065603,0.0,0.000000,0.000000,2.623775
12,40080000,60990000,24,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,364.380158,359.428455,210.546497,0.056135,1795.953379,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,100.815417,111.809734,0.0,60990000,C,1795.953379,1.0,46.740642,2.623775,2.623775
33,40390000,61540000,30,C,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,97.576377,115.832831,64.096011,0.027324,500.849234,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.685065,13.685065,0.0,61540000,C,500.849234,1.0,108.382317,2.961408,2.961408
1,40420000,60530000,19,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,23.326241,40.567376,26.368794,0.102284,154.065603,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.758478,135.053492,0.0,60530000,C,154.065603,0.0,0.000000,0.000000,5.089941
51,40420000,60710000,30,C,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,117.816891,152.888385,89.978360,0.027324,614.865324,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,16.800409,135.053492,0.0,60710000,C,614.865324,1.0,90.257320,2.466166,5.089941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87713,831380000,60850103,29,A,57.428416,72.207652,135.459522,194.409719,141.793750,318.526897,569.348679,564.134832,403.377763,0.030807,2663.613353,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,82.059031,5941.468440,0.0,60850103,A,2663.613353,0.0,0.000000,0.000000,7.491266
49230,831380000,60950101,19,A,77.910796,87.987431,210.729551,255.894450,155.595176,478.030563,686.597900,648.683485,415.708340,0.102284,3209.257599,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,328.256368,5941.468440,0.0,60950101,A,3209.257599,0.0,0.000000,0.000000,7.491266
55000,831380000,60950102,22,A,67.975915,77.960010,155.903079,226.090400,145.568264,401.767992,533.594615,490.010580,296.409159,0.071361,2514.314725,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,179.424691,5941.468440,0.0,60950102,A,2514.314725,6.0,22.124378,1.578824,7.491266
54896,831380000,60950103,20,A,46.976690,68.925850,125.728002,224.736092,103.186116,259.487677,570.055119,581.257106,330.447800,0.090718,2514.398596,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,228.101094,5941.468440,0.0,60950103,A,2514.398596,0.0,0.000000,0.000000,7.491266


In [22]:
apl_final = od[od["IRIS1"] == od["IRIS2"]]
apl_final.head(10)

,IRIS1,IRIS2,TIME,TYP_IRIS,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4559,pop6074,pop75p,WexpGP,POPGP,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,b,un,weight,NB,wpop,swpop,R,IRIS1_dest,TYP_IRIS_dest,POPGP_dest,NB_dest,R_dest,Ap,APL
385,60010000,60010000,0,C,4.000000,4.000000,9.000000,1.000000,2.000000,15.000000,19.000000,23.000000,9.000000,1.0,92.810629,60010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,92.810629,278.379262,0.000000,60010000,C,92.810629,0.000000,0.000000,0.000000,34.838750
157,60020000,60020000,0,C,0.944444,0.000000,0.944444,0.000000,4.722222,2.833333,15.111111,15.111111,9.444444,1.0,57.033492,60020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,57.033492,236.380068,0.000000,60020000,C,57.033492,0.000000,0.000000,0.000000,0.000000
242,60030000,60030000,0,C,9.614569,20.984124,45.775888,60.317381,36.232745,93.143833,151.790455,144.847089,33.296336,1.0,622.897211,60030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,622.897211,1206.589178,0.000000,60030000,C,622.897211,0.000000,0.000000,0.000000,17.915476
74748,60040101,60040101,0,C,52.615008,37.899626,73.326247,64.895515,185.630635,461.058851,400.386503,279.922874,69.647193,1.0,1644.085142,60040101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1644.085142,174437.803372,0.000000,60040101,C,1644.085142,0.000000,0.000000,0.000000,152.380745
74398,60040102,60040102,0,A,112.596525,83.873253,181.151638,248.450983,129.235637,410.866823,530.224938,416.764450,243.638581,1.0,2409.644871,60040102,16425.0,DAEMERS,YVES,6.0,92989.0,7.072160,43.596934,16425_43.596934_7.07216,1.0,1.0,2.000000,2409.644871,139332.759307,1.435413,60040102,A,2409.644871,2.000000,1.435413,1.435413,121.937806
74130,60040103,60040103,0,A,18.241996,10.944370,16.186080,30.114241,46.846024,97.925058,215.575652,162.194344,175.037444,1.0,864.660366,60040103,113924.0,DIAMANT,GUY,6.0,77467.0,7.137517,43.567521,113924_43.567521_7.137517,1.0,1.0,1.000000,864.660366,78645.292998,1.271532,60040103,A,864.660366,1.000000,1.271532,1.271532,74.016465
83097,60040104,60040104,0,A,52.059060,62.638659,106.336802,200.433608,72.021065,291.013376,447.263183,496.404417,280.639329,1.0,2172.216717,60040104,16404.0,MANDRILE,DIDIER,6.0,1651.0,7.092388,43.604533,16404_43.604533_7.092388,1.0,1.0,1.000000,2172.216717,163639.281073,0.611100,60040104,A,2172.216717,1.000000,0.611100,0.611100,143.539160
73476,60040105,60040105,0,A,121.696271,114.291373,153.996744,184.875202,193.965266,724.551827,692.494861,860.079264,622.608181,1.0,3992.283201,60040105,16222.0,NICASTRO,SERGE,6.0,92975.0,7.117538,43.588105,16222_43.588105_7.117538,1.0,1.0,2.000000,3992.283201,135683.050817,1.474023,60040105,A,3992.283201,2.000000,1.474023,1.474023,118.345411
73130,60040106,60040106,0,A,75.577264,67.518049,89.334063,114.362122,88.077558,404.229484,512.572858,610.280247,480.319986,1.0,2719.305357,60040106,144024.0,FRICKEY,SARAH,6.0,2734.0,7.112369,43.606074,144024_43.606074_7.112369,1.0,1.0,8.666667,2719.305357,131229.252582,6.604219,60040106,A,2719.305357,8.666667,6.604219,6.604219,117.825191
72782,60040107,60040107,0,A,118.067240,69.100801,149.669616,166.439662,172.098492,755.118545,584.505561,551.536589,342.120410,1.0,3038.758340,60040107,17085.0,RAYBAUD,GUILLAUME,6.0,99691.0,7.121421,43.600414,17085_43.600414_7.121421,2.0,0.5,4.000000,3038.758340,139694.151980,2.863398,60040107,A,3038.758340,4.000000,2.863398,2.863398,123.704089


In [23]:
apl_final["APL"].describe() #mean=98 median=108 du 2205 au 2305, median=110 en 21, pour pa median=62

count    500.000000
mean     103.068905
std       54.664702
min        0.000000
25%       66.487800
50%      109.311151
75%      144.807338
max      517.141914
Name: APL, dtype: float64

In [24]:
apl_final = apl_final[["IRIS1", "TYP_IRIS", "NB", "APL"]]
apl_final.head(10)

,IRIS1,TYP_IRIS,NB,APL
385,60010000,C,0.000000,34.838750
157,60020000,C,0.000000,0.000000
242,60030000,C,0.000000,17.915476
74748,60040101,C,0.000000,152.380745
74398,60040102,A,2.000000,121.937806
74130,60040103,A,1.000000,74.016465
83097,60040104,A,1.000000,143.539160
73476,60040105,A,2.000000,118.345411
73130,60040106,A,8.666667,117.825191
72782,60040107,A,4.000000,123.704089


In [25]:
sql = f"""
select i.id "IRIS1", i.code "IRIS", i.nom "LIB_IRIS", c.dept_id "DEP", c.code "DEPCOM", c.nom "LIBCOM", 20{year} "YEAR", {specialite} "SPECIALITE" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '{dept:02d}%'"""
print(sql)
nom_iris = pd.read_sql(text(sql), config.connection_string)
# nom_iris = pd.read_stata("Etude 06/NomIRIS.dta")
nom_iris



select i.id "IRIS1", i.code "IRIS", i.nom "LIB_IRIS", c.dept_id "DEP", c.code "DEPCOM", c.nom "LIBCOM", 2021 "YEAR", 10 "SPECIALITE" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '06%'


,IRIS1,IRIS,LIB_IRIS,DEP,DEPCOM,LIBCOM,YEAR,SPECIALITE
0,60120105,060120105,Centre,6,06012,Beausoleil,2021,10
1,60500000,060500000,Coursegoules,6,06050,Coursegoules,2021,10
2,60720000,060720000,Ilonse,6,06072,Ilonse,2021,10
3,61050101,061050101,Nord,6,06105,Roquefort-les-Pins,2021,10
4,60150000,060150000,Berre-les-Alpes,6,06015,Berre-les-Alpes,2021,10
...,...,...,...,...,...,...,...,...
495,60883403,060883403,Madeleine Supérieure,6,06088,Nice,2021,10
496,60883501,060883501,Crémat-Bellet,6,06088,Nice,2021,10
497,60883601,060883601,Saint-Pierre de Féric,6,06088,Nice,2021,10
498,60883602,060883602,Costière,6,06088,Nice,2021,10


In [26]:
apl_final = apl_final.merge(nom_iris, on="IRIS1", how="left", suffixes=("", "_dest"))
apl_final = apl_final[["IRIS", "TYP_IRIS", "NB", "APL", "LIB_IRIS", "DEP", "DEPCOM", "LIBCOM", "YEAR", "SPECIALITE"]]
apl_final

,IRIS,TYP_IRIS,NB,APL,LIB_IRIS,DEP,DEPCOM,LIBCOM,YEAR,SPECIALITE
0,060010000,C,0.0,34.838750,Aiglun,6,06001,Aiglun,2021,10
1,060020000,C,0.0,0.000000,Amirat,6,06002,Amirat,2021,10
2,060030000,C,0.0,17.915476,Andon,6,06003,Andon,2021,10
3,060040101,C,0.0,152.380745,Les Trois Moulins,6,06004,Antibes,2021,10
4,060040102,A,2.0,121.937806,Les Semboules,6,06004,Antibes,2021,10
...,...,...,...,...,...,...,...,...,...,...
495,061610105,A,5.0,159.555523,Marina,6,06161,Villeneuve-Loubet,2021,10
496,061610106,A,0.0,115.572944,Vaugrenier,6,06161,Villeneuve-Loubet,2021,10
497,061610107,A,0.0,127.306723,Plan,6,06161,Villeneuve-Loubet,2021,10
498,061620000,C,0.5,66.751294,La Brigue,6,06162,La Brigue,2021,10


In [27]:
apl_final[apl_final["IRIS"]=='061290000']

,IRIS,TYP_IRIS,NB,APL,LIB_IRIS,DEP,DEPCOM,LIBCOM,YEAR,SPECIALITE
436,061290000,C,2.5,517.141914,Saint-Sauveur-sur-Tinée,6,06129,Saint-Sauveur-sur-Tinée,2021,10


In [28]:
# TODO trouver les ratio DAMIR
# TODO remplacer ps par personne_activite : prévoir les 2